In [449]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from datetime import datetime
import calendar 

import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)



In [61]:
import src.utils.preprocessing as pre

# Methods

## Preprocessing

In [210]:
#Two different Formats identified in the DAT column with typos
#Error handling code from Stacksoverflow: "validate for multiple date / datetime formats" by 'c8999c 3f964f64'
def validate_datetime(string, whitelist=('%H:%M, %d %B %Y', '%H:%M, %d %b %Y')):
    for fmt in whitelist:
        try:
            dt = datetime.strptime(string, fmt)
        except ValueError:
            pass
        else: # if a defined format is found, datetime object will be returned
            return dt
    else: # all formats done, none did work...
        return False # could also raise an exception here
    
def dates_prep(data_original):

    data_original['YEA'] = data_original['YEA'].astype(int)

    dates =[]
    for t in data_original['DAT']:
        dates.append(validate_datetime(t))

    dates = pd.DataFrame(dates)
    dates[dates[0]==False] = dates[dates[0]==False].replace(to_replace=False, value = float('NaN'))

    datetime_object_date = dates.apply(lambda x: x[0].date() if type(x[0])!=float else x[0], axis=1)
    datetime_object_month= dates.apply(lambda x: calendar.month_name[x[0].month] if type(x[0])!=float else x[0], axis=1)
    datetime_object_time = dates.apply(lambda x: x[0].time() if type(x[0])!=float else x[0], axis=1)

    # Converting types of DAT and adding TIM column
    data_original['DAT'] = datetime_object_date
    data_original['MON'] = datetime_object_month
    data_original['TIM'] = datetime_object_time
    # Reordering columns
    data_original = data_original.iloc[:,[0,1,2,3,4,5,7,8,6]]

    return data_original

def prep_unique_elections(data_original):
    uniq_elections = data_original.drop_duplicates(subset=['TGT','YEA','RES']).groupby('TGT').value_counts(dropna=False).reset_index()
    uniq_elections= uniq_elections.drop('count', axis=1)

    uniq_cand_freq_table= pd.DataFrame(uniq_elections['TGT'].value_counts())
    sing_runners_list = uniq_cand_freq_table[uniq_cand_freq_table['count']==1].index
    mult_runners_list = uniq_cand_freq_table[uniq_cand_freq_table['count']>1].index
    
    uniq_cand_freq_table = uniq_cand_freq_table.reset_index()

    return uniq_elections, uniq_cand_freq_table, sing_runners_list, mult_runners_list

def assign_elec_id(uniq_elections_data, complete_dataset): #i.e. (unique_elections_data, data)
    uniq_elections_data['elec_id']= np.linspace(1,len(uniq_elections_data),len(uniq_elections_data))
    uniq_elections_data['elec_id']= uniq_elections_data['elec_id'].astype(int)
    
    complete_dataset['elec_id'] = np.empty(198275)
    complete_dataset['elec_id'] = complete_dataset['elec_id'].astype(int)

    for row in range(0,len(complete_dataset['TGT']),1):
        complete_dataset.iloc[row,9] = uniq_elections_data[(uniq_elections_data['TGT']==complete_dataset.iloc[row,1]) & 
                                        (uniq_elections_data['YEA']== complete_dataset.iloc[row,4]) &
                                        (uniq_elections_data['RES']== complete_dataset.iloc[row,3])]['elec_id']
    #print(data.iloc[row,[0,1,9]], row) #To Troubleshoot and find row which is giving error

    return uniq_elections_data, complete_dataset


def calc_win_loss(uniq_cand_freq_table, uniq_elections_data):

    win,loss =[],[]
    
    for row in range(0,len(uniq_cand_freq_table),1):
        if len(uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts().index) == 2:
            loss.append(uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts()[0])
            win.append(uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts()[1])
        elif len(uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts().index) ==1 and\
        uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts().index[0][1] == -1:
            loss.append(uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts()[0])
            win.append(0)
        elif len(uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts().index) ==1 and\
        uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts().index[0][1] == 1:
            loss.append(0)
            win.append(uniq_elections_data[uniq_elections_data['TGT']==uniq_cand_freq_table.iloc[row,0]][['TGT','RES']].value_counts()[0])
        else:
            print('Something unexpected happen')

    uniq_cand_freq_table['win'] = win
    uniq_cand_freq_table['loss'] = loss
    return uniq_cand_freq_table

def flag_elec_id(uniq_elections_data, complete_dataset):

    flagged = []

    for elec_id in uniq_elections_data['elec_id']:
        #check if same month, next or previous month as mode
        for idx in complete_dataset[complete_dataset['elec_id']==elec_id].index:
            if type(complete_dataset.iloc[idx,5]) == float:
                #print('skipped NaN')
                continue

            elif (datetime.strptime(complete_dataset[complete_dataset['elec_id']==elec_id]['MON'].mode()[0], '%B').month) == 1:
                if (datetime.strptime(complete_dataset[complete_dataset['elec_id']==elec_id]['MON'].mode()[0], '%B').month == complete_dataset.iloc[idx,5].month) or \
                    (12 == complete_dataset.iloc[idx,5].month) or \
                    (2 == complete_dataset.iloc[idx,5].month):
                    pass
                    #print('No Problem - Jan', idx)

            elif (datetime.strptime(complete_dataset[complete_dataset['elec_id']==elec_id]['MON'].mode()[0], '%B').month) == 12:
                if (datetime.strptime(complete_dataset[complete_dataset['elec_id']==elec_id]['MON'].mode()[0], '%B').month == complete_dataset.iloc[idx,5].month) or \
                    (11 == complete_dataset.iloc[idx,5].month) or \
                    (1 == complete_dataset.iloc[idx,5].month):
                    pass
                    #print('No Problem- Dec', idx)

            elif (datetime.strptime(complete_dataset[complete_dataset['elec_id']==elec_id]['MON'].mode()[0], '%B').month) != 1 and 2:
                if (datetime.strptime(complete_dataset[complete_dataset['elec_id']==elec_id]['MON'].mode()[0], '%B').month == complete_dataset.iloc[idx,5].month) or \
                    ((datetime.strptime(complete_dataset[complete_dataset['elec_id']==elec_id]['MON'].mode()[0], '%B').month+1) == complete_dataset.iloc[idx,5].month) or \
                    ((datetime.strptime(complete_dataset[complete_dataset['elec_id']==elec_id]['MON'].mode()[0], '%B').month-1) == complete_dataset.iloc[idx,5].month):
                    pass
                    #print('No Problem', idx)
                else:
                    #print('row', idx)
                    flagged.append([elec_id, idx])
    
    flagged = pd.DataFrame(flagged, columns=('elec_id','index'))

    return flagged

def get_flagged_boundaries(flagged):
    flagged_with_boundaries = (flagged.groupby('elec_id').min()).merge(flagged.groupby('elec_id').max(), on='elec_id', suffixes=('_lower','_upper'))

    return flagged_with_boundaries

def separate_flagged_elec_id(complete_dataset, flagged):
    
    x = complete_dataset.copy()
    flagged_with_bounds = get_flagged_boundaries(flagged=flagged)

    for elec_id in flagged_with_bounds.index:
        try:    
            x[x['elec_id']==elec_id].loc[flagged_with_bounds.loc[elec_id]['index_lower']-1][9]
            #print('flagged election is after, shift elec_id from lower bound')
            
            # Case 1: flagged election is after, shift elec_id from lower bound
            for idx in x.index:
                if x.iloc[idx,9]> elec_id:
                    x.iloc[idx,9] = x.iloc[idx,9]+1

            for idx in range(flagged_with_bounds.index_lower.loc[elec_id], flagged_with_bounds.index_upper.loc[elec_id]+1,1):
                x.iloc[idx,9] = x.iloc[idx,9]+1


        except KeyError:
            #print('flagged election is before, shift elec_id above upper bound')

            # Case 2: flagged election is before, shift elec_id above upper bound
            for idx in x.index:
                if x.iloc[idx,9]>= elec_id:
                    x.iloc[idx,9] = x.iloc[idx,9]+1

            for idx in range(flagged_with_bounds.index_lower.loc[elec_id], flagged_with_bounds.index_upper.loc[elec_id]+1,1):
                x.iloc[idx,9] = x.iloc[idx,9]-1
        
    file_path = 'data/processed_elec_data.csv'
    x.to_csv(file_path, index=False)

    return x

## Votes 

In [211]:
def get_votes(complete_dataset):

    votes_by_elec = pd.DataFrame(complete_dataset.groupby('elec_id')['RES'].agg('count'))
    votes_by_elec = votes_by_elec.rename(columns={'RES':'TOT'})
    votes_by_elec = votes_by_elec.reset_index()

    pos = complete_dataset[complete_dataset['VOT']==1].groupby('elec_id')['VOT'].agg('count').reset_index()
    pos = pos.rename(columns={'VOT':'POS'})
    #pos

    abs = complete_dataset[complete_dataset['VOT']==0].groupby('elec_id')['VOT'].agg('count').reset_index()
    abs = abs.rename(columns={'VOT':'ABS'})
    #abs

    neg = complete_dataset[complete_dataset['VOT']==-1].groupby('elec_id')['VOT'].agg('count').reset_index()
    neg = neg.rename(columns={'VOT':'NEG'})
    #abs

    votes_by_elec = votes_by_elec.merge(pos, on='elec_id', how='left')
    votes_by_elec = votes_by_elec.merge(abs, on='elec_id', how='left')
    votes_by_elec = votes_by_elec.merge(neg, on='elec_id', how='left')

    votes_by_elec= votes_by_elec.fillna(0)


    votes_by_elec['POS'] = votes_by_elec['POS'].astype(int)
    votes_by_elec['ABS'] = votes_by_elec['ABS'].astype(int)
    votes_by_elec['NEG'] = votes_by_elec['NEG'].astype(int)

    complete_dataset = complete_dataset.merge(votes_by_elec, how='left',on='elec_id')

    return votes_by_elec, complete_dataset

# Code

In [63]:
data = pre.import_RFA()

print(data.shape)
data.head(5)

(198275, 7)


,SRC,TGT,VOT,RES,YEA,DAT,TXT
0,Steel1943,BDD,1,1,2013,"23:13, 19 April 2013",Support as conom.
1,Cuchullain,BDD,1,1,2013,"01:04, 20 April 2013",Support as nominator.
2,INeverCry,BDD,1,1,2013,"23:43, 19 April 2013",Support per noms.
3,Cncmaster,BDD,1,1,2013,"00:11, 20 April 2013",Support per noms. BDD is a strong contributor ...
4,Miniapolis,BDD,1,1,2013,"00:56, 20 April 2013","Support, with great pleasure. I work with BDD ..."


In [64]:
df = dates_prep(data)
df

,SRC,TGT,VOT,RES,YEA,DAT,MON,TIM,TXT
0,Steel1943,BDD,1,1,2013,2013-04-19,April,23:13:00,Support as conom.
1,Cuchullain,BDD,1,1,2013,2013-04-20,April,01:04:00,Support as nominator.
2,INeverCry,BDD,1,1,2013,2013-04-19,April,23:43:00,Support per noms.
3,Cncmaster,BDD,1,1,2013,2013-04-20,April,00:11:00,Support per noms. BDD is a strong contributor ...
4,Miniapolis,BDD,1,1,2013,2013-04-20,April,00:56:00,"Support, with great pleasure. I work with BDD ..."
...,...,...,...,...,...,...,...,...,...
198270,172,Vancouverguy,1,1,2003,2003-09-02,September,02:51:00,Support
198271,Angela,WhisperToMe,1,1,2003,2003-11-26,November,23:45:00,Support.
198272,Jiang,WhisperToMe,1,1,2003,NaN,NaN,NaN,Support.
198273,Pakaran,WhisperToMe,1,1,2003,2003-12-05,December,05:38:00,Support. Age has nothing to do with maturity.


In [65]:
unique_elections, unique_candidate_freq_table, single_runners_list, multiple_runners_list  = prep_unique_elections(df)


In [66]:
#unique_elections
#unique_candidate_freq_table
#single_runners_list
#multiple_runners_list

In [67]:
unique_elections, df = assign_elec_id(unique_elections,df)

C:\Users\mikai\AppData\Local\Temp\ipykernel_24580\3499867910.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_dataset['elec_id'] = np.empty(198275)
C:\Users\mikai\AppData\Local\Temp\ipykernel_24580\3499867910.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  complete_dataset['elec_id'] = complete_dataset['elec_id'].astype(int)


In [68]:
#unique_elections
#df

In [69]:
unique_candidate_freq_table = calc_win_loss(unique_candidate_freq_table,unique_elections)

In [32]:
#unique_candidate_freq_table

In [33]:
flagged_elec_id = flag_elec_id(unique_elections, df)

In [34]:
flagged_elec_id

,elec_id,index
0,81,27420
1,81,27421
2,81,27422
3,81,27423
4,81,27424
...,...,...
4272,3892,120135
4273,3892,120136
4274,3892,120137
4275,3892,120138


In [35]:
#Ran Once, long runtime
#df_processed = separate_flagged_elec_id(df,flagged_elec_id)

In [36]:
#importing csv mentioned in previous cell
file_path = 'data/processed_elec_data.csv'
df_processed = pd.read_csv(file_path)

# Votes

In [212]:
df1 = df.copy()
df1

,SRC,TGT,VOT,RES,YEA,DAT,MON,TIM,TXT,elec_id
0,Steel1943,BDD,1,1,2013,2013-04-19,April,23:13:00,Support as conom.,301
1,Cuchullain,BDD,1,1,2013,2013-04-20,April,01:04:00,Support as nominator.,301
2,INeverCry,BDD,1,1,2013,2013-04-19,April,23:43:00,Support per noms.,301
3,Cncmaster,BDD,1,1,2013,2013-04-20,April,00:11:00,Support per noms. BDD is a strong contributor ...,301
4,Miniapolis,BDD,1,1,2013,2013-04-20,April,00:56:00,"Support, with great pleasure. I work with BDD ...",301
...,...,...,...,...,...,...,...,...,...,...
198270,172,Vancouverguy,1,1,2003,2003-09-02,September,02:51:00,Support,3558
198271,Angela,WhisperToMe,1,1,2003,2003-11-26,November,23:45:00,Support.,3655
198272,Jiang,WhisperToMe,1,1,2003,NaN,NaN,NaN,Support.,3655
198273,Pakaran,WhisperToMe,1,1,2003,2003-12-05,December,05:38:00,Support. Age has nothing to do with maturity.,3655


In [213]:
votes, df1 = get_votes(df1)

In [214]:
df1

,SRC,TGT,VOT,RES,YEA,DAT,MON,TIM,TXT,elec_id,TOT,POS,ABS,NEG
0,Steel1943,BDD,1,1,2013,2013-04-19,April,23:13:00,Support as conom.,301,137,119,4,14
1,Cuchullain,BDD,1,1,2013,2013-04-20,April,01:04:00,Support as nominator.,301,137,119,4,14
2,INeverCry,BDD,1,1,2013,2013-04-19,April,23:43:00,Support per noms.,301,137,119,4,14
3,Cncmaster,BDD,1,1,2013,2013-04-20,April,00:11:00,Support per noms. BDD is a strong contributor ...,301,137,119,4,14
4,Miniapolis,BDD,1,1,2013,2013-04-20,April,00:56:00,"Support, with great pleasure. I work with BDD ...",301,137,119,4,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198270,172,Vancouverguy,1,1,2003,2003-09-02,September,02:51:00,Support,3558,3,3,0,0
198271,Angela,WhisperToMe,1,1,2003,2003-11-26,November,23:45:00,Support.,3655,3,3,0,0
198272,Jiang,WhisperToMe,1,1,2003,NaN,NaN,NaN,Support.,3655,3,3,0,0
198273,Pakaran,WhisperToMe,1,1,2003,2003-12-05,December,05:38:00,Support. Age has nothing to do with maturity.,3655,3,3,0,0


# Graphs

In [283]:
sing_mult_stats = pd.DataFrame({'SING' : unique_elections[unique_elections['TGT'].isin(single_runners_list)]['RES'].value_counts(normalize=False),
              'SING_PERC' : unique_elections[unique_elections['TGT'].isin(single_runners_list)]['RES'].value_counts(normalize=True),
              'MULT' : unique_elections[unique_elections['TGT'].isin(multiple_runners_list)]['RES'].value_counts(normalize=False),
              'MULT_PERC' : unique_elections[unique_elections['TGT'].isin(multiple_runners_list)]['RES'].value_counts(normalize=True)
})

sing_mult_stats_total = pd.DataFrame(sing_mult_stats.sum(axis=0)).rename(columns={0:'Total'}).transpose().astype(int)

sing_mult_stats = pd.concat([sing_mult_stats,sing_mult_stats_total], axis=0)
sing_mult_stats

,SING,SING_PERC,MULT,MULT_PERC
-1,1454,0.467374,550,0.674847
1,1657,0.532626,265,0.325153
Total,3111,1.000000,815,1.000000


In [394]:
passrate_by_year = pd.DataFrame({'WIN': df1.drop_duplicates('elec_id').reset_index().drop('index', axis=1).groupby('YEA')['RES'].value_counts(normalize=False).unstack().fillna(0).iloc[:,1].astype(int),
                                 'LOSS': df1.drop_duplicates('elec_id').reset_index().drop('index', axis=1).groupby('YEA')['RES'].value_counts(normalize=False).unstack().fillna(0).iloc[:,0].astype(int),
                                 'WIN_PERC': df1.drop_duplicates('elec_id').reset_index().drop('index', axis=1).groupby('YEA')['RES'].value_counts(normalize=True).unstack().fillna(0).iloc[:,1],
                                 'LOSS_PERC': df1.drop_duplicates('elec_id').reset_index().drop('index', axis=1).groupby('YEA')['RES'].value_counts(normalize=True).unstack().fillna(0).iloc[:,0],
})

passrate_by_year.reset_index()

passrate_by_year

,WIN,LOSS,WIN_PERC,LOSS_PERC
YEA,,,,
2003,58,0,1.000000,0.000000
2004,236,58,0.802721,0.197279
2005,380,185,0.672566,0.327434
2006,347,472,0.423687,0.576313
2007,406,443,0.478210,0.521790
2008,201,332,0.377111,0.622889
2009,120,209,0.364742,0.635258
2010,75,141,0.347222,0.652778
2011,52,83,0.385185,0.614815


## Graph 1: Single vs. Multiple Time Runners Pass Rate

In [460]:
df_g =sing_mult_stats.drop(['SING_PERC','MULT_PERC'],axis =1).transpose().reset_index().rename(columns={'index':'type', -1:'neg', 1:'pos'})
df_g2 =sing_mult_stats.drop(['SING','MULT'],axis =1).transpose().reset_index().rename(columns={'index':'type', -1:'neg', 1:'pos'})
df_g2


fig = go.Figure(
    data=[
        go.Bar(x=df_g.type, y=df_g.neg, name="Losses", text=df_g.neg, textposition='inside'),
        go.Bar(x=df_g.type, y=df_g.pos, name="Wins", text=df_g.pos, textposition=['inside','outside']),
        go.Bar(x=df_g.type, y=df_g.Total, name="Total", text=df_g.Total, textposition='inside'),
    ],
    layout=dict(
        barcornerradius=15,
    ),
)

# Overlay line graph for Pass Rate, centered on "Wins" bars
fig.add_trace(
    go.Scatter(
        x=df_g.type,  # Use the same x values as the "Wins" bar
        y=[pr * 100 for pr in df_g2["pos"]],  # Convert pass rate to percentage
        mode="lines+markers+text",  # Line graph with markers and text
        name="Pass Rate",
        text=[f"{pr*100:.1f}%" for pr in df_g2["pos"]],  # Add pass rate as percentage text
        textposition="top center",
        line=dict(color="red", width=2),
        marker=dict(color="red", size=8),
        yaxis="y2",  # Associate this trace with the secondary y-axis
    )
)

# Update layout to add secondary y-axis
fig.update_layout(
    title=dict(
        text="Candidate Pass Rates: One Time vs Multiple Times",  # Chart title
        x=0.5,  # Center the title
        xanchor="center",  # Anchor the title at the center
    ),
    xaxis=dict(
        title="Times Candidate Participated",
        tickvals=["SING", "MULT"],  # Define custom labels
        ticktext=["Candidate Only Once", "Candidate Multiple Times"],
    ),
    yaxis=dict(title="Counts", side="left"),
    yaxis2=dict(
        title="Pass Rate (%)",
        overlaying="y",
        side="right",
        range=[0, 100],
        tickformat=".0f%",
    ),
    barmode="group",
    legend=dict(
        title="Legend",
        x=1.15,
        y=0.6,
        xanchor="right",
        yanchor="middle",
    ),
)

pio.write_html(fig, file="Plots/pass_rates_once_v_multiple.html", auto_open=False)


fig.show()


## Graph 2: Election Statistics over time

In [ ]:
df_g = passrate_by_year.drop(['WIN_PERC','LOSS_PERC'],axis =1)
df_g['TOTAL'] = df_g.sum(axis=1)
df_g = df_g.reset_index()

df_g2 = passrate_by_year.drop(['WIN','LOSS'],axis =1).reset_index().rename(columns={'index':'type'})

fig = go.Figure(
    data=[
        go.Bar(x=df_g.YEA, y=df_g.LOSS, name="LOSS", text=df_g.LOSS, textposition='inside'),
        go.Bar(x=df_g.YEA, y=df_g.WIN, name="WIN", text=df_g.WIN, textposition=['inside','outside']),
        go.Bar(x=df_g.YEA, y=df_g.TOTAL, name="Total", text=df_g.TOTAL, textposition='inside'),
    ],
    layout=dict(
        barcornerradius=15,
    ),
)

fig.update_traces(
    textfont=dict(color="white"),  # Set text color to white
    selector=dict(type="bar")  # Apply only to bar traces
)

# Overlay line graph for Pass Rate, centered on "Wins" bars
fig.add_trace(
    go.Scatter(
        x=df_g.YEA,  # Use the same x values as the "Wins" bar
        y=[pr * 100 for pr in df_g2["WIN_PERC"]],  # Convert pass rate to percentage
        mode="lines+markers+text",  # Line graph with markers and text
        name="Pass Rate",
        text=[f"{pr*100:.1f}%" for pr in df_g2["WIN_PERC"]],  # Add pass rate as percentage text
        textposition=['bottom center',"top center","top center","top center","top center","top center","top center","top center","top center","top center","top center"],
        line=dict(color="red", width=2),
        marker=dict(color="red", size=8),
        yaxis="y2",  # Associate this trace with the secondary y-axis
    )
)

# Update layout to add secondary y-axis
fig.update_layout(
    title=dict(
        text="Elections and Win Rates Across The Years",  # Chart title
        x=0.5,  # Center the title
        xanchor="center",  # Anchor the title at the center
    ),
    xaxis=dict(
        title="Year",
        tickvals=df_g.YEA,
        
    ),
    yaxis=dict(title="Counts", side="left"),
    yaxis2=dict(
        title="Pass Rate (%)",
        overlaying="y",
        side="right",
        range=[0, 100],
        tickformat=".0f%",
    ),
    barmode="group",
    legend=dict(
        title="Legend",
        x=1.15,
        y=0.6,
        xanchor="right",
        yanchor="middle",
    ),
)


pio.write_html(fig, file="Plots/Evolution_of_Pass_Rates_and_votes.html", auto_open=False)

fig.show()


## Graph 3: Vote Distribution by election outcome over time

In [596]:

df_g = pd.DataFrame(df1.groupby('RES')[['YEA','RES','TOT']].value_counts())
df_g = df_g.reset_index()
df_g = df_g.drop('count', axis=1)

year = 2003
rows = 4
cols = 3

# Create subplot layout with titles for each subplot
subplot_titles = [f"Year {year + i}" for i in range(12)]
subplot_titles[-1] = ""  # Hide the title of the last plot

fig = make_subplots(rows = rows, cols = cols, subplot_titles=subplot_titles)

color_pairs = [
    ("rgba(135, 206, 250, 0.8)", "rgba(255, 165, 0, 0.8)"),  # Light blue & orange
    ("rgba(147, 112, 219, 0.8)", "rgba(0, 255, 127, 0.8)"),  # Purple & teal
    ("rgba(173, 255, 47, 0.8)", "rgba(255, 105, 180, 0.8)"),  # Green & pink
]


for idx in range(0,11,1):

    row = (idx // cols) + 1  # Integer division to determine row
    col = (idx % cols) + 1   # Modulo to determine column

        # Alternate colors based on the index
    loss_color, win_color = color_pairs[idx % len(color_pairs)]

    # Add Histograms for RES=1 and RES=-1
    fig.add_trace(
        go.Histogram(x=df_g[(df_g['YEA'] == year + idx) & (df_g['RES'] == 1)]['TOT'], 
                     name='Win',
                     marker_color= win_color,
                     showlegend =False
                     ),
        row=row,
        col=col,
    )
    fig.add_trace(
        go.Histogram(x=df_g[(df_g['YEA'] == year + idx) & (df_g['RES'] == -1)]['TOT'], 
                     name='Loss',
                     marker_color = loss_color,
                     showlegend =False
                     ),
        row=row,
        col=col,
    )


# Update layout with global settings
fig.update_layout(
    height=1000,       # Adjust the height
    width=1600,       # Adjust the width
    
)

# Set common x-axis and y-axis labels
fig.update_xaxes(title_text="Total Count", row=rows, col=2)  # Bottom row x-axis
fig.update_yaxes(title_text="Frequency", row=2, col=1)       # First column y-axis

# Common x-axis range
common_xrange = [0, 350]  # Replace with the appropriate range if known
fig.update_xaxes(range=common_xrange)

# Common y-axis range
common_yrange = [0, 50]  # Replace with the appropriate range if known
fig.update_yaxes(range=common_yrange)

# Remove the last plot by hiding its axes
fig.update_xaxes(visible=False, row=rows, col=cols)
fig.update_yaxes(visible=False, row=rows, col=cols)

fig.update_layout(barmode='overlay', title =dict(
        text="Distribution of Votes by Election Outcome over the Years",  # Chart title
        x=0.5,  # Center the title
        xanchor="center",  # Anchor the title at the center
    )
)
fig.update_traces(opacity=0.8)


pio.write_html(fig, file="Plots/dist_votes_by_elec_outcome_over_years.html", auto_open=False)

fig.show()

- Steady shift of the mass towards the right for Wins --> more and more votes needed to win elections
- Mass of rejections stays mostly anchored under 100 votes, but frequency fluctuates a lot --> low election participation is still a eliminating factor but very it's usage fluctuates

# Graph 4: Voting Behavior, Pass Rates, and Votes Per Election

In [909]:
df_g = df1.drop_duplicates('elec_id')[['YEA','TOT','POS','ABS','NEG']]
df_g = df_g.reset_index()
df_g = df_g.drop('index', axis=1)
df_g = df_g.groupby('YEA').sum().reset_index()

df_g['POS_pct'] = (df_g['POS'] / df_g['TOT']) * 100
df_g['ABS_pct'] = (df_g['ABS'] / df_g['TOT']) * 100
df_g['NEG_pct'] = (df_g['NEG'] / df_g['TOT']) * 100


df_g2 = passrate_by_year.drop(['WIN','LOSS'],axis =1).reset_index().rename(columns={'index':'type'})

df_g3 = df1.drop_duplicates('elec_id').groupby('YEA')[['elec_id']].count().rename(columns={'elec_id':'no_elec'})
df_g3 = df_g3.reset_index()
df_g3['VOT_PER_ELEC'] = df_g['TOT']/df_g3['no_elec']

# Melt the DataFrame for plotting
df_melted = df_g.melt(
    id_vars=['YEA', 'TOT'], 
    value_vars=['POS', 'ABS', 'NEG'], 
    var_name='Type', 
    value_name='Value'
)

# Add percentages for text
df_melted['Percentage'] = df_melted.apply(
    lambda row: (row['Value'] / df_g.loc[df_g['YEA'] == row['YEA'], 'TOT'].values[0]) * 100, axis=1
)

# Create the figure
fig = go.Figure()

# Add the bar chart with percentage text
for type_name in df_melted['Type'].unique():
    filtered_df = df_melted[df_melted['Type'] == type_name]
    fig.add_trace(
        go.Bar(
            x=filtered_df['YEA'],
            y=filtered_df['Value'],
            name=type_name,
            text=[f"{p:.1f}%" for p in filtered_df['Percentage']],  # Use percentage as text
            textposition='inside'
        )
    )

fig.update_traces(
    textfont=dict(color="white"),  # Set text color to white
    selector=dict(type="bar")  # Apply only to bar traces
)

# Add the scatter plot for pass rates
fig.add_trace(
    go.Scatter(
        x=df_g['YEA'],  # Use the same x values
        y=[pr * 100 for pr in df_g2["WIN_PERC"]],  # Convert pass rate to percentage
        mode="lines+markers+text",  # Line graph with markers and text
        name="Pass Rate",
        text=[f"{pr*100:.1f}%" for pr in df_g2["WIN_PERC"]],  # Add pass rate as percentage text
        textposition='bottom center',
        line=dict(color="red", width=2),
        marker=dict(color="red", size=8),
        yaxis="y2"  # Associate this trace with the secondary y-axis
    )
)

fig.update_traces(
    textfont=dict(color="red"),  # Set text color to white
    selector=dict(type="scatter")  # Apply only to bar traces
)

# Configure layout
fig.update_layout(
    barmode='stack',
    title=dict(
        text="Voting Behavior and Pass Rates over a Decade",  # Chart title
        x=0.5,  # Center the title
        xanchor="center",  # Anchor the title at the center
    ),
    xaxis=dict(
        title="Year",
        tickmode='linear',  # Force all ticks to be shown
        tick0=2003,         # Starting point
        dtick=1             # Interval of 1 year
    ),
    yaxis=dict(title="Total Count"),
    yaxis2=dict(
        title="Pass Rate (%)",
        overlaying="y",
        side="right",  # Place the secondary y-axis on the right
        range=[0, 100]  # Set the y-axis scale from 0 to 100
    ),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=800
)

fig.add_trace(
    go.Scatter(
        x=df_g3['YEA'],  # Year for x-axis
        y=[50000] * len(df_g3),  # Fixed y-value to align bubbles on the same line
        mode="markers+text",  # Scatter with markers and text
        name="Votes Per Election",
        text=[f"{v:.0f}" for v in df_g3['VOT_PER_ELEC']],  # Display value inside the bubble
        textposition="top center",  # Position text above the bubbles
        marker=dict(
            size=df_g3['VOT_PER_ELEC'] *1,  # Scale bubble size for visual impact
            color="blue",
            opacity=0.7
        ),
        hoverinfo="text+x",  # Show year and votes per election in hover
        hovertext=[f"Year: {year}<br>Votes Per Election: {votes:.0f}" for year, votes in zip(df_g3['YEA'], df_g3['VOT_PER_ELEC'])]
    )
)

# Update layout to ensure bubbles fit nicely
fig.update_layout(
    title=dict(
        text="Voting Behavior, Pass Rates, and Votes Per Election",
        x=0.5,
        xanchor="center"
    ),
    xaxis=dict(
        title="Year",
        tickmode='array',  # Use an array to specify which ticks to show
        tickvals=[2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013],  # Exclude 2014
        ticktext=["2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013"]  # Text for these ticks
    ),
    yaxis=dict(
        title="Total Count"
    ),
    yaxis2=dict(
        title="Pass Rate (%)",
        overlaying="y",
        side="right",
        range=[0, 120],  # Ensure the secondary y-axis is from 0 to 100
        tickvals=[0, 20, 40, 60, 80, 100],  # Define ticks you want to display
        ticktext=["0%", "20%", "40%", "60%", "80%", "100%"]  # Custom text for these ticks
    ),
    height=800,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

pio.write_html(fig, file="Plots/voting_beh_passrates_vot_per_elec.html", auto_open=False)

fig.show()